
<a id = '4.0'></a>
<p style = "font-size : 35px; color : #34656d ;  text-align : center; background-color : #08f9f9; border-radius: 5px 5px;"><strong>Logistic Regression (Diabetes)</strong></p> 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#%matplotlib inline
#%matplotlib notebook
plt.rcParams["figure.figsize"] = (10,6)
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")
pd.set_option('display.float_format', lambda x: '%.3f' % x)
#pd.options.display.float_format = '{:.3f}'.format

In [ ]:
df=pd.read_csv("diabetes.csv")

In [ ]:
df.head()

##### DiabetesPedigreeFunction: diabetes history in relatives
##### BMI: Body mass index

In [ ]:
df.shape

## Exploratory Data Analysis and Visualization

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.Outcome.value_counts()

In [ ]:
sns.countplot(df.Outcome);

In [ ]:
sns.boxplot(df.Pregnancies);

In [ ]:
#df=df[df.Pregnancies<13]

In [ ]:
sns.boxplot(df.Pregnancies);

In [ ]:
sns.boxplot(df.SkinThickness);

In [ ]:
df=df[df.SkinThickness<70]

In [ ]:
sns.boxplot(df.SkinThickness);

In [ ]:
sns.boxplot(df.Insulin);

In [ ]:
sns.boxplot(df.Glucose);

In [ ]:
df=df[df.Glucose>0]

In [ ]:
sns.boxplot(df.Glucose);

In [ ]:
sns.boxplot(df.BloodPressure);

In [ ]:
df=df[df.BloodPressure>35]

In [ ]:
sns.boxplot(df.BloodPressure);

In [ ]:
sns.boxplot(df.BMI);

In [ ]:
df=df[df.BMI>0]

In [ ]:
sns.boxplot(df.BMI);

In [ ]:
df.shape

In [ ]:
df.Outcome.value_counts()

In [ ]:
index = 0
plt.figure(figsize=(20,20))
for feature in df.columns:
    if feature != "Outcome":
        index += 1
        plt.subplot(3,3,index)
        sns.boxplot(x='Outcome',y=feature,data=df)

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(), annot=True);

In [ ]:
# df.corr()
# df.corr()["Outcome"].sort_values().plot.barh()
df.corr()["Outcome"].drop("Outcome").sort_values().plot.barh();

In [ ]:
sns.pairplot(df, hue = "Outcome");

## Train | Test Split and Scaling

In [ ]:
X=df.drop(["Outcome"], axis=1)
y=df["Outcome"]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=42) # stratify is....

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Modelling

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_model=LogisticRegression()

In [ ]:
log_model.fit(X_train_scaled, y_train)

In [ ]:
y_pred=log_model.predict(X_test_scaled)

In [ ]:
y_pred_proba = log_model.predict_proba(X_test_scaled)

In [ ]:
test_data = pd.concat([X_test, y_test], axis=1)
test_data["pred"] = y_pred
test_data["pred_proba"] = y_pred_proba[:,1]
test_data.sample(10)

## Model Performance on Classification Tasks

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
def eval_metric(model, X_train, y_train, X_test, y_test):
    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_test)
    
    print("Test_Set")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print()
    print("Train_Set")
    print(confusion_matrix(y_train, y_train_pred))
    print(classification_report(y_train, y_train_pred))

In [ ]:
eval_metric(log_model, X_train_scaled, y_train, X_test_scaled, y_test)

## Cross Validate

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
model = LogisticRegression()

scores = cross_validate(model, X_train_scaled, y_train, scoring = ['precision','recall','f1','accuracy'], cv = 10)
df_scores = pd.DataFrame(scores, index = range(1, 11))
df_scores

In [ ]:
df_scores.mean()[2:]

In [ ]:
eval_metric(log_model, X_train_scaled, y_train, X_test_scaled, y_test)

## Cross Validate for 0 class

In [ ]:
import sklearn
sklearn.metrics.SCORERS.keys()

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

f1_0 = make_scorer(f1_score, pos_label =0)
precision_0 = make_scorer(precision_score, pos_label =0)
recall_0 = make_scorer(recall_score, pos_label =0)

In [ ]:
model = LogisticRegression()

scores = cross_validate(model, X_train_scaled, y_train, scoring = {"precision_0":precision_0, "recall_0":recall_0, "f1_0":f1_0}, 
                        cv = 10)
df_scores = pd.DataFrame(scores, index = range(1, 11))
df_scores

In [ ]:
df_scores.mean()[2:]

In [ ]:
eval_metric(log_model, X_train_scaled, y_train, X_test_scaled, y_test)

## GridSearchCV

In [ ]:
import sklearn
sklearn.metrics.SCORERS.keys()

In [ ]:
from sklearn.model_selection import GridSearchCV

model = LogisticRegression()

penalty = ["l1", "l2"]
C = np.logspace(-1, 5, 20)
class_weight= ["balanced", None] 
# The "balanced" mode uses the values of y to automatically adjust weights inversely proportional to class frequencies 
# in the input data
solver = ["lbfgs", "liblinear", "sag", "saga"]

param_grid = {"penalty" : penalty,
              "C" : C,
              "class_weight":class_weight,
              "solver":solver}


grid_model = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=10,
                          scoring = "recall",   #f1_0 = make_scorer(f1_score, pos_label =0)
                          n_jobs = -1)

In [ ]:
grid_model.fit(X_train_scaled,y_train)

In [ ]:
grid_model.best_params_

In [ ]:
eval_metric(grid_model, X_train_scaled, y_train, X_test_scaled, y_test)

## ROC (Receiver Operating Curve) and AUC (Area Under Curve)

https://towardsdatascience.com/calculating-and-setting-thresholds-to-optimise-logistic-regression-performance-c77e6d112d7e

In [ ]:
from sklearn.metrics import plot_roc_curve, plot_precision_recall_curve, roc_auc_score, auc, roc_curve, average_precision_score, precision_recall_curve

In [ ]:
plot_roc_curve(grid_model, X_test_scaled, y_test);

In [ ]:
plot_precision_recall_curve(grid_model, X_test_scaled, y_test);

## Finding Best Threshold for max recall score

In [ ]:
plot_roc_curve(log_model, X_train_scaled, y_train);

In [ ]:
y_pred_proba = log_model.predict_proba(X_train_scaled) #
roc_auc_score(y_train, y_pred_proba[:,1])

In [ ]:
fp_rate, tp_rate, thresholds = roc_curve(y_train, y_pred_proba[:,1])

In [ ]:
optimal_idx = np.argmax(tp_rate - fp_rate)
optimal_threshold = thresholds[optimal_idx]
optimal_threshold

In [ ]:
plot_precision_recall_curve(log_model, X_train_scaled, y_train);

In [ ]:
y_pred_proba = log_model.predict_proba(X_train_scaled)
average_precision_score(y_train, y_pred_proba[:,1])

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(y_train, y_pred_proba[:,1])

In [ ]:
optimal_idx = np.argmax((2 * precisions * recalls) / (precisions + recalls))
optimal_threshold = thresholds[optimal_idx]
optimal_threshold

In [ ]:
grid_model.predict_proba(X_test_scaled)[:,1]

In [ ]:
y_pred2 = pd.Series(grid_model.predict_proba(X_test_scaled)[:,1]).apply(lambda x : 1 if x >= optimal_threshold else 0)

In [ ]:
print(confusion_matrix(y_test,y_pred2))
print(classification_report(y_test,y_pred2))

In [ ]:
y_train_pred2 = pd.Series(grid_model.predict_proba(X_train_scaled)[:,1]).apply(lambda x : 1 if x >= optimal_threshold else 0)
print(confusion_matrix(y_train, y_train_pred2))
print(classification_report(y_train, y_train_pred2))

## Final Model and Model Deployment

In [ ]:
scaler = StandardScaler().fit(X)

In [ ]:
import pickle
pickle.dump(scaler, open("scaler_diabates", 'wb'))

In [ ]:
X_scaled = scaler.transform(X)

In [ ]:
final_model = LogisticRegression(C= 0.1, class_weight= 'balanced',penalty= 'l1',solver= 'liblinear').fit(X_scaled, y)

In [ ]:
pickle.dump(final_model, open("final_model_diabates", 'wb'))

In [ ]:
X.describe().T

# Using the Model

In [ ]:
my_dict = {"Pregnancies": [3, 6, 5],
           "Glucose": [117, 140, 120],
           "BloodPressure": [72, 80, 75],
           "SkinThickness": [23, 33, 25],
           "Insulin": [48, 132, 55],
           "BMI": [32, 36.5, 34],
           "DiabetesPedigreeFunction": [0.38, 0.63, 0.45],
           "Age": [29, 40, 33]
          }

In [ ]:
sample = pd.DataFrame(my_dict)
sample

In [ ]:
scaler_diabates = pickle.load(open("scaler_diabates", "rb"))

In [ ]:
sample_scaled = scaler_diabates.transform(sample)
sample_scaled

In [ ]:
final_model = pickle.load(open("final_model_diabates", "rb"))

In [ ]:
predictions = final_model.predict(sample_scaled)
predictions_proba = final_model.predict_proba(sample_scaled)
predictions2 = [1 if i >= optimal_threshold else 0 for i in predictions_proba[:,1]]

In [ ]:
sample["pred_proba"] = predictions_proba[:,1]
sample["pred_0.50"] = predictions
sample["pred_0.34"] = predictions2
sample